# Извлечение признаков и балансировка

In [1]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

def extract_features(file_path, duration=1, offset=0.25):
    features = []
    audio, sr = librosa.load(file_path, sr=None, duration=duration, offset=offset)
    
    num_segments = int(np.ceil(len(audio) / (sr * duration)))
    
    for i in range(num_segments):
        start = int(sr * duration * i)
        end = int(min(len(audio), sr * duration * (i + 1)))
        
        segment = audio[start:end] 
        
        mfccs = librosa.feature.mfcc(y=segment, sr=sr)
        chroma = librosa.feature.chroma_stft(y=segment, sr=sr)
        mel = librosa.feature.melspectrogram(y=segment, sr=sr)
        contrast = librosa.feature.spectral_contrast(y=segment, sr=sr, fmin=100.0, n_bands=6)
        
        features.append(np.mean(mfccs, axis=1))
        features.append(np.mean(chroma, axis=1))
        features.append(np.mean(mel, axis=1))
        features.append(np.mean(contrast, axis=1))
    
    if len(features) > 0:
        return np.concatenate(features)
    else:
        return None


data_dir = '/Users/midasxlr/Desktop/DroneAudioDataset-master'

features = []
labels = []

duration = 1  # длина окна в секундах
offset = 0.5  # шаг прохода окна в секундах

for root, dirs, files in os.walk(data_dir):
    base_dir = os.path.basename(root)
    if base_dir in ["yes_drone", "bebop_1", "membo_1","Micro"]:
        label = 1  # Звук дрона
    elif base_dir == "unknown":
        label = 0
        
        
    for file in files:
        if file.endswith(".wav"):
            file_path = os.path.join(root, file)
            feature = extract_features(file_path, duration, offset)
            if feature is not None:
                features.append(feature)
                labels.append(label)

                
X = np.array(features)
y = np.array(labels)

# Разбиение на тренировочный и тестовый наборы данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=108)

# Создание объекта RandomUnderSampler для undersampling
under_sampler = RandomUnderSampler(sampling_strategy='majority', random_state=108)

# Применение undersampling к тренировочным данным
X_resampled_under, y_resampled_under = under_sampler.fit_resample(X_train, y_train)

# Создание объекта RandomOverSampler для oversampling
over_sampler = RandomOverSampler(sampling_strategy='minority', random_state=108)

# Применение oversampling к данным после undersampling
X_resampled, y_resampled = over_sampler.fit_resample(X_resampled_under, y_resampled_under)

# Обучение модели на несбалансированных данных
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = model.predict(X_test)

# Оценка точности на тестовом наборе данных
accuracy = accuracy_score(y_test, y_pred)
print("Точность модели на тестовых данных:", accuracy)

# Вывод отчета о классификации
print("\nОтчет о классификации на несбалансированных данных:")
print(classification_report(y_test, y_pred))

# Обучение модели на сбалансированных данных
model_resampled = xgb.XGBClassifier()
model_resampled.fit(X_resampled, y_resampled)

# Предсказание на тестовых данных
y_pred_resampled = model_resampled.predict(X_test)

# Оценка точности на тестовом наборе данных
accuracy_resampled = accuracy_score(y_test, y_pred_resampled)
print("\nТочность модели на тестовых данных сбалансированными данными:", accuracy_resampled)

# Вывод отчета о классификации
print("\nОтчет о классификации на сбалансированных данных:")
print(classification_report(y_test, y_pred_resampled))


/Users/midasxlr/anaconda3/lib/python3.11/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=478
  warnings.warn(
/Users/midasxlr/anaconda3/lib/python3.11/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/var/folders/7m/qr8d586122zcnb2d14b1c1480000gn/T/ipykernel_70095/3894666072.py:13: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file_path, sr=None, duration=duration, offset=offset)
/Users/midasxlr/anaconda3/lib/python3.11/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Точность модели на тестовых данных: 0.9989411266412537

Отчет о классификации на несбалансированных данных:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4152
           1       1.00      0.99      1.00       570

    accuracy                           1.00      4722
   macro avg       1.00      1.00      1.00      4722
weighted avg       1.00      1.00      1.00      4722


Точность модели на тестовых данных сбалансированными данными: 0.9896230410842863

Отчет о классификации на сбалансированных данных:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      4152
           1       0.92      1.00      0.96       570

    accuracy                           0.99      4722
   macro avg       0.96      0.99      0.98      4722
weighted avg       0.99      0.99      0.99      4722



# Создание и сохранение модели

In [2]:
import xgboost as xgb

model = xgb.XGBClassifier()
model.fit(X_train, y_train)

model_path = '/Users/midasxlr/Desktop/New_side_model0.5'
model.save_model(model_path)

print("Модель сохранена по пути:", model_path)

Модель сохранена по пути: /Users/midasxlr/Desktop/New_side_model0.5


/Users/midasxlr/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [20:00:49] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
